***Participantes (RM - NOME):***<br>
345377 - Alison Junior Ghedin<br>
345946 - Leonardo Sarzedas do Carmo Vieira<br>
344526 - Michael Douglas Barbosa Araujo<br>


###**Criar um classificador de chamados aplicando técnicas de PLN**
---

A **DinDinAgora** tem um canal de atendimento via chat e precisar classificar os assuntos dos atendimentos para melhorar as tratativas dos chamados dos clientes. O canal recebe textos abertos dos clientes relatando o problema e/ou dúvida e depois é direcionado para algum uma área especialista no assunto para uma melhor tratativa.​

Crie um modelo classificador de assuntos aplicando técnicas de PLN, que consiga classificar através de um texto o assunto conforme disponível na base de dados [1] para treinamento e validação do modelo seu modelo.​

O modelo precisar atingir um score na **métrica F1 Score superior a 75%**. Utilize o dataset [1] para treinar e testar o modelo, separe o dataset em duas amostras (75% para treinamento e 25% para teste com o randon_state igual a 42).​

Fique à vontade para testar e explorar as técnicas de pré-processamento, abordagens de NLP, algoritmos e bibliotecas, mas explique e justifique as suas decisões durante o desenvolvimento.​

**Composição da nota:​**

**50%** - Demonstrações das aplicações das técnicas de PLN (regras, pré-processamentos, tratamentos, variedade de modelos aplicados, organização do pipeline, etc.)​

**50%** - Baseado na performance (score) obtida com a amostra de teste no pipeline do modelo campeão (validar com  a Métrica F1 Score). **Separar o pipeline completo do modelo campeão conforme template.​**

**[1] = ​https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv**

Obs.: Para a métrica F1 Score, usar o parâmetro average = 'weighted'.

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html

In [1]:
# instalações de pacotes
!pip install spacy
!python -m spacy download pt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-03-27 22:32:58.861550: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-27 22:33:01.411128: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-27 22:33:01.411286: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nv

In [2]:
#import de bibliotecas
import spacy
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split

import re
import spacy
import nltk



In [3]:
# CARREGANDO O DATA FRAME
df = pd.read_csv('https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv', delimiter=';')

# Façam o download do arquivo e utilizem localmente durante os testes

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21072 entries, 0 to 21071
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id_reclamacao         21072 non-null  int64 
 1   data_abertura         21072 non-null  object
 2   categoria             21072 non-null  object
 3   descricao_reclamacao  21072 non-null  object
dtypes: int64(1), object(3)
memory usage: 658.6+ KB


Bom desenvolvimento!

#Verificando se há desbalanceamento das bases

In [5]:
df.categoria.value_counts()

Serviços de conta bancária             5161
Cartão de crédito / Cartão pré-pago    5006
Roubo / Relatório de disputa           4822
Hipotecas / Empréstimos                3850
Outros                                 2233
Name: categoria, dtype: int64

# Balanceando a base de dados

In [6]:
# Resample da amostra
sample_df = df.sample(frac=1,random_state=4)
outros_df = sample_df.loc[sample_df.categoria=="Outros"]
conta_df = sample_df.loc[sample_df.categoria=="Serviços de conta bancária"].sample(n=outros_df.shape[0], random_state=42)
roubo_df = sample_df.loc[sample_df.categoria=="Roubo / Relatório de disputa"].sample(n=outros_df.shape[0], random_state=42)
cartao_df = sample_df.loc[sample_df.categoria=="Cartão de crédito / Cartão pré-pago"].sample(n=outros_df.shape[0], random_state=42)
emprestimos_df = sample_df.loc[sample_df.categoria=="Hipotecas / Empréstimos"].sample(n=outros_df.shape[0], random_state=42)




In [7]:
df.categoria.value_counts()

Serviços de conta bancária             5161
Cartão de crédito / Cartão pré-pago    5006
Roubo / Relatório de disputa           4822
Hipotecas / Empréstimos                3850
Outros                                 2233
Name: categoria, dtype: int64

In [8]:
df = pd.concat([outros_df, conta_df, roubo_df, cartao_df, emprestimos_df])
df = df.sample(frac=1,random_state=42)
df.head()

,id_reclamacao,data_abertura,categoria,descricao_reclamacao
12438,2477745,2017-05-07T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,Isso se refere à placa CHAS E/ XXXX XXXX XXXX ...
2292,4188418,2021-03-06T12:00:00-05:00,Roubo / Relatório de disputa,"Em xx/xx/2020, comprometi -me a comprar uma má..."
10569,1502439,2015-08-04T12:00:00-05:00,Outros,Violação da Lei da Verdade em Empréstimos/Regu...
20805,2834040,2018-03-05T12:00:00-05:00,Outros,A edição do JP Morgan Chase envolve dinheiro e...
18835,2125959,2016-09-22T12:00:00-05:00,Serviços de conta bancária,Meu marido recebeu um visto de recompensa do A...


In [9]:
df.reset_index(drop=True, inplace=True)

In [10]:
df

,id_reclamacao,data_abertura,categoria,descricao_reclamacao
0,2477745,2017-05-07T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,Isso se refere à placa CHAS E/ XXXX XXXX XXXX ...
1,4188418,2021-03-06T12:00:00-05:00,Roubo / Relatório de disputa,"Em xx/xx/2020, comprometi -me a comprar uma má..."
2,1502439,2015-08-04T12:00:00-05:00,Outros,Violação da Lei da Verdade em Empréstimos/Regu...
3,2834040,2018-03-05T12:00:00-05:00,Outros,A edição do JP Morgan Chase envolve dinheiro e...
4,2125959,2016-09-22T12:00:00-05:00,Serviços de conta bancária,Meu marido recebeu um visto de recompensa do A...
...,...,...,...,...
11160,2142135,2016-10-03T12:00:00-05:00,Roubo / Relatório de disputa,Um cartão de crédito emitido pelo Chase Bank d...
11161,3341890,2019-08-07T12:00:00-05:00,Roubo / Relatório de disputa,Entrei em contato com o Chase Bank sobre meu i...
11162,3585631,2020-03-30T12:00:00-05:00,Roubo / Relatório de disputa,Transferi um saldo {$ 5000.00} para o meu cart...
11163,3939092,2020-11-05T12:00:00-05:00,Outros,CHASE AUTO CONTA NÚMERO XXXX XXXX XXXX As decl...


In [11]:
df.shape

(11165, 4)

In [12]:
df.categoria.value_counts()

Cartão de crédito / Cartão pré-pago    2233
Roubo / Relatório de disputa           2233
Outros                                 2233
Serviços de conta bancária             2233
Hipotecas / Empréstimos                2233
Name: categoria, dtype: int64

# Verificar o texto e o que poderia ser filtrado

In [13]:
df.descricao_reclamacao[1]

'Em xx/xx/2020, comprometi -me a comprar uma máquina de corte XXXX XXXX usada em xxxx (item # xxxx,) e o processo de checkout xxxx utilizado para enviar o pagamento com meu cartão de crédito Chase. O valor da transação, incluindo frete, era {$ 1000,00}. O vendedor enviou um pacote contendo um tijolo, lavanderia suja e lixo e forneceu um número de rastreamento xxxx. Quando o pacote chegou ao nosso endereço comercial, sua entrega foi confirmada pela assinatura. Depois que o driver XXXX deixou, abri a embalagem e descobri que ele não continha a máquina de corte xxxx xxxx e senti que houve uma tentativa deliberada do vendedor de me fraudar.\r\n\r\nEm xx/xx/2020, enviei uma disputa via xxxx e selecionei a opção que não recebi o item. Na minha opinião, essa foi absolutamente uma declaração verdadeira, pois não recebi o item que me comprometi a comprar. Infelizmente, isso fez com que o sistema de resolução XXXX faça referência ao número de rastreamento xxxx fornecido pelo scammer e regra a se

# Load da biblioteca spacy

In [14]:
nlp = spacy.load("pt_core_news_sm")

# Função de limpeza de texto removendo:
pontuação, quebra de linha, informações sensíveis, valores, acentuação

In [15]:
#remover o XXXX
def limpar_textos(texto):
  texto = re.sub(r'[,.:;]', '', texto)
  texto = re.sub(r'\n', ' ', texto)
  texto = re.sub(r'[xxxx]', '', texto)
  texto = re.sub(r'[XXXX]', '', texto)
  texto = re.sub(r'[xx/xx/xxxx]', '', texto)
  texto = re.sub(r'\{.*?\}', '', texto)
  texto = re.sub('\\s+', ' ', texto)
  texto = re.sub(u'[^a-zA-ZéúíóáÉÚÍÓÁèùìòàÈÙÌÒÀõãñÕÃÑêûîôâÊÛÎÔÂëÿüïöäËYÜÏÖÄçÇ ]', '', texto)
  texto = texto.strip().lower()
  return texto

# Funções de lematização de texto completo e de verbos

In [16]:
# função de lematização completa do documento
def lematiza_texto(texto: str):
  sent = []
  doc = nlp(texto)
  for word in doc:
    sent.append(word.lemma_)
  return " ".join(sent)

# função de lematização de verbos do documento
def lematiza_verbos(texto:str):
  sent = []
  doc = nlp(texto)
  for word in doc:
    if word.pos_ == "VERB":
      sent.append(word.lemma_)
    else:
      sent.append(word.text)
  return " ".join(sent)


In [17]:
df["text_treat"] = df.descricao_reclamacao.apply(limpar_textos)

In [18]:
df.text_treat.head()

0    isso se refere à placa chas e eu me inscrevi n...
1    em  comprometi me a comprar uma máquina de cor...
2    violação da lei da verdade em empréstimosregul...
3    a edição do jp morgan chase envolve dinheiro e...
4    meu marido recebeu um visto de recompensa do a...
Name: text_treat, dtype: object

# Criar uma amostra dos dados para uma execução de testes mais rápida.

In [19]:
df_raw, df_amostra = train_test_split(df, 
                                       test_size=0.1, 
                                       random_state=42)

# Lemmatização de texto

In [20]:
df_amostra["text_lemma"] = df_amostra.text_treat.apply(lematiza_texto)

# Lemmatização de verbos

In [21]:
df_amostra["text_lemma_verbos"] = df_amostra.text_treat.apply(lematiza_verbos)

# Separação de amostra entre treino e teste

In [22]:
# divisão das amostras em treino e teste
df_treino, df_teste = train_test_split(df_amostra, 
                                       test_size=0.3, 
                                       random_state=42)

# Teste 1
StopWords: NLTK e Spacy<br> 
Vetorização: 
  - TFid<br> 
  - Frequencia dos termos invertida<br>
  - Ngram: Unigrama<br> 
  - Norm: l2<br> 
Lematização:
  - Texto  
Modelo:
  - Regressão Logistica

In [23]:
nltk.download('stopwords')
nlp = spacy.load('pt_core_news_sm')

stops_spacy = nlp.Defaults.stop_words
stops_nltk = nltk.corpus.stopwords.words('portuguese')
stops = list(set(stops_spacy).union(set(stops_nltk)))

vetor_tfid = TfidfVectorizer(ngram_range=(1,1), use_idf=True, stop_words=stops, norm='l2')
vetor_tfid.fit(df_treino.text_lemma)
text_vect_treino = vetor_tfid.transform(df_treino.text_lemma)

linear_model = LogisticRegression(random_state=42, class_weight='balanced')
linear_model.fit(text_vect_treino, df_treino["categoria"])

text_vect_test = vetor_tfid.transform(df_teste.text_lemma)
pred = linear_model.predict(text_vect_test)
print(f"F1 Score: {f1_score(df_teste['categoria'], pred, average='weighted')}")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


F1 Score: 0.8756101045316731


# Teste 2
StopWords: Spacy<br> 
Vetorização: 
  - TFid<br> 
  - Ngram: combinação Unigrama/Bigrama<br> 
  - Norm: l1
Lematização:
  - Verbos  
Modelo:
  - Regressão Logistica

In [24]:
vetor_tfid = TfidfVectorizer(ngram_range=(1,2), use_idf=False, stop_words=list(stops_spacy), norm='l1')
vetor_tfid.fit(df_treino.text_lemma)
text_vect_treino = vetor_tfid.transform(df_treino.text_lemma)

linear_model = LogisticRegression(random_state=42, class_weight='balanced')
linear_model.fit(text_vect_treino, df_treino["categoria"])

text_vect_test = vetor_tfid.transform(df_teste.text_lemma)
pred = linear_model.predict(text_vect_test)

accuracy = accuracy_score(pred, df_teste['categoria'])
print(f"F1 Score: {f1_score(df_teste['categoria'], pred, average='weighted')}")

F1 Score: 0.8071385375630659


# Teste 3
StopWords: NLTK<br> 
Vetorização: 
  - TFid<br> 
  - Ngram: Unigrama<br> 
  - Norm: l2<br> 
Lematização:
  - Texto  
Modelo:
  - Regressão Logistica

In [25]:
vetor_tfid = TfidfVectorizer(ngram_range=(1,1), use_idf=False, stop_words=stops_nltk, norm='l2')
vetor_tfid.fit(df_treino.text_lemma)
text_vect_treino = vetor_tfid.transform(df_treino.text_lemma)

linear_model = LogisticRegression(random_state=42, class_weight='balanced')
linear_model.fit(text_vect_treino, df_treino["categoria"])

text_vect_test = vetor_tfid.transform(df_teste.text_lemma)
pred = linear_model.predict(text_vect_test)

print(f"F1 Score: {f1_score(df_teste['categoria'], pred, average='weighted')}")

F1 Score: 0.8606966029219033


# Teste 4
StopWords: NLTK e Spacy<br> 
Vetorização: 
  - CountVectorizer<br> 
  - Ngram: Unigrama<br> 
Lematização:
  - Texto  
Modelo:
  - Regressão Logistica

In [26]:
vect = CountVectorizer(ngram_range=(1,1), stop_words=stops)
vect.fit(df_treino.text_lemma)
text_vect_treino = vect.transform(df_treino.text_lemma)

linear_model = LogisticRegression(random_state=42, class_weight='balanced')
linear_model.fit(text_vect_treino, df_treino["categoria"])

text_vect_test = vect.transform(df_teste.text_lemma)
pred = linear_model.predict(text_vect_test)

print(f"F1 Score: {f1_score(df_teste['categoria'], pred, average='weighted')}")

F1 Score: 0.856248209989192


# Teste 5
StopWords: NLTK e Spacy<br> 
Vetorização: 
  - CountVectorizer<br> 
  - Ngram: Unigrama<br> 
Lematização:
  - Verbos<br>
Modelo:
  - Regressão Logistica

In [27]:
vect = CountVectorizer(ngram_range=(1,1), stop_words=stops)
vect.fit(df_treino.text_lemma_verbos)
text_vect_treino = vect.transform(df_treino.text_lemma_verbos)

linear_model = LogisticRegression(random_state=42, class_weight='balanced')
linear_model.fit(text_vect_treino, df_treino["categoria"])

text_vect_test = vect.transform(df_teste.text_lemma_verbos)
pred = linear_model.predict(text_vect_test)

accuracy = accuracy_score(pred, df_teste['categoria'])
print(f"F1 Score: {f1_score(df_teste['categoria'], pred, average='weighted')}")

F1 Score: 0.8537029660223244


# Teste 6
StopWords: NLTK e Spacy<br> 
Vetorização: 
  - CountVectorizer<br> 
  - Ngram: Unigrama<br> 
Lematização:
  - Verbos<br>
Modelo:
  - Decision Tree

In [28]:
vect = CountVectorizer(ngram_range=(1,1), stop_words=stops)
vect.fit(df_treino.text_lemma_verbos)
text_vect_treino = vect.transform(df_treino.text_lemma_verbos)

tree_model = DecisionTreeClassifier()
tree_model.fit(text_vect_treino, df_treino["categoria"])

text_vect_test = vect.transform(df_teste.text_lemma_verbos)
pred = tree_model.predict(text_vect_test)

print(f"F1 Score: {f1_score(df_teste['categoria'], pred, average='weighted')}")

F1 Score: 0.6568159449246891


# Teste 7
StopWords: NLTK e Spacy<br> 
Vetorização: 
  - CountVectorizer<br> 
  - Ngram: Combinação Unigrama/Bigrama<br> 
Lematização:
  - Verbos<br>
Modelo:
  - Decision Tree

In [29]:
vect = CountVectorizer(ngram_range=(1,2), stop_words=stops)
vect.fit(df_treino.text_lemma_verbos)
text_vect_treino = vect.transform(df_treino.text_lemma_verbos)

tree_model = DecisionTreeClassifier()
tree_model.fit(text_vect_treino, df_treino["categoria"])

text_vect_test = vect.transform(df_teste.text_lemma_verbos)
pred = tree_model.predict(text_vect_test)

print(f"F1 Score: {f1_score(df_teste['categoria'], pred, average='weighted')}")

F1 Score: 0.6700091451207523


# Teste 8
StopWords: NLTK e Spacy<br> 
Vetorização: 
  - CountVectorizer<br> 
  - Ngram: Unigrama<br> 
Lematização:
  - Texto<br>
Modelo:
  - Decision Tree

In [30]:
vect = CountVectorizer(ngram_range=(1,1), stop_words=stops)
vect.fit(df_treino.text_lemma)
text_vect_treino = vect.transform(df_treino.text_lemma)

tree_model = DecisionTreeClassifier()
tree_model.fit(text_vect_treino, df_treino["categoria"])

text_vect_test = vect.transform(df_teste.text_lemma)
pred = tree_model.predict(text_vect_test)

print(f"F1 Score: {f1_score(df_teste['categoria'], pred, average='weighted')}")

F1 Score: 0.7156348444826439


# Teste 9
StopWords: NLTK e Spacy<br> 
Vetorização: 
  - CountVectorizer<br> 
  - Ngram: Unigrama<br> 
Lematização:
  - Texto<br>
Modelo:
  - Random Forest

In [31]:
vect = CountVectorizer(ngram_range=(1,1), stop_words=stops)
vect.fit(df_treino.text_lemma)
text_vect_treino = vect.transform(df_treino.text_lemma)

trees_model = RandomForestClassifier()
trees_model.fit(text_vect_treino, df_treino["categoria"])

text_vect_test = vect.transform(df_teste.text_lemma)
pred = trees_model.predict(text_vect_test)


print(f"F1 Score: {f1_score(df_teste['categoria'], pred, average='weighted')}")

F1 Score: 0.7892377685519111


# Teste 10
StopWords: NLTK e Spacy<br> 
Vetorização: 
  - CountVectorizer<br> 
  - Ngram: Combinação Unigrama/Bigrama<br> 
Lematização:
  - Texto<br>
Modelo:
  - Random Forest

In [32]:
vect = CountVectorizer(ngram_range=(1,2), stop_words=stops)
vect.fit(df_treino.text_lemma)
text_vect_treino = vect.transform(df_treino.text_lemma)

trees_model = RandomForestClassifier()
trees_model.fit(text_vect_treino, df_treino["categoria"])

text_vect_test = vect.transform(df_teste.text_lemma)
pred = trees_model.predict(text_vect_test)

print(f"F1 Score: {f1_score(df_teste['categoria'], pred, average='weighted')}")

F1 Score: 0.801241975793109


# Teste 11
StopWords: NLTK e Spacy<br> 
Vetorização: 
  - TFid<br> 
  - Frequencia dos termos invertida<br>
  - Ngram: Unigrama<br> 
  - Norm: l2<br> 
Lematização:
  - Verbos  
Modelo:
  - Random Forest

In [33]:
vetor_tfid = TfidfVectorizer(ngram_range=(1,1), use_idf=True, stop_words=stops, norm='l2')

vetor_tfid.fit(df_treino.text_lemma_verbos)
text_vect_treino = vetor_tfid.transform(df_treino.text_lemma_verbos)

trees_model = RandomForestClassifier()
trees_model.fit(text_vect_treino, df_treino["categoria"])

text_vect_test = vetor_tfid.transform(df_teste.text_lemma_verbos)
pred = trees_model.predict(text_vect_test)

print(f"F1 Score: {f1_score(df_teste['categoria'], pred, average='weighted')}")

F1 Score: 0.8074431573818996


# Teste 12
StopWords: NLTK<br> 
Vetorização: 
  - TFid<br> 
  - Frequencia dos termos invertida<br>
  - Ngram: Combinação Unigrama/Bigrama<br> 
  - Norm: l2<br> 
Lematização:
  - Texto  
Modelo:
  - Regressão Logistica

In [34]:
nltk.download('stopwords')
stops_nltk = nltk.corpus.stopwords.words('portuguese')

vetor_tfid = TfidfVectorizer(ngram_range=(1,2), use_idf=True, stop_words=list(stops_nltk), norm='l2')
vetor_tfid.fit(df_treino.text_lemma)
text_vect_treino = vetor_tfid.transform(df_treino.text_lemma)


model = LogisticRegression(random_state=42, class_weight='balanced')
model.fit(text_vect_treino, df_treino["categoria"])

text_vect_test = vetor_tfid.transform(df_teste.text_lemma)
pred = model.predict(text_vect_test)

print(f"F1 Score: {f1_score(df_teste['categoria'], pred, average='weighted')}")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


F1 Score: 0.8990351159424108


####**Validação do professor**

Consolidar apenas os scripts do seu **modelo campeão**, desde o carregamento do dataframe, separação das amostras, tratamentos utilizados (funções, limpezas, etc.), criação dos objetos de vetorização dos textos e modelo treinado e outras implementações utilizadas no processo de desenvolvimento do modelo.

# Instalação de pacotes

In [1]:
# instalações de pacotes
!pip install spacy
!python -m spacy download pt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-03-27 22:36:53.328829: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-27 22:36:55.656674: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-27 22:36:55.656791: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/

# Import de bibliotecas

In [2]:
import pandas as pd
import spacy
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split

import re
import nltk


# Carregar a base de dados

In [3]:
df = pd.read_csv('https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv', delimiter=';')

# Resample da base de dados

In [4]:
sample_df = df.sample(frac=1,random_state=4)
outros_df = sample_df.loc[sample_df.categoria=="Outros"]
conta_df = sample_df.loc[sample_df.categoria=="Serviços de conta bancária"].sample(n=outros_df.shape[0], random_state=42)
roubo_df = sample_df.loc[sample_df.categoria=="Roubo / Relatório de disputa"].sample(n=outros_df.shape[0], random_state=42)
cartao_df = sample_df.loc[sample_df.categoria=="Cartão de crédito / Cartão pré-pago"].sample(n=outros_df.shape[0], random_state=42)
emprestimos_df = sample_df.loc[sample_df.categoria=="Hipotecas / Empréstimos"].sample(n=outros_df.shape[0], random_state=42)
df = pd.concat([outros_df, conta_df, roubo_df, cartao_df, emprestimos_df])
df = df.sample(frac=1,random_state=42)
df.reset_index(drop=True, inplace=True)
df.head()

,id_reclamacao,data_abertura,categoria,descricao_reclamacao
0,2477745,2017-05-07T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,Isso se refere à placa CHAS E/ XXXX XXXX XXXX ...
1,4188418,2021-03-06T12:00:00-05:00,Roubo / Relatório de disputa,"Em xx/xx/2020, comprometi -me a comprar uma má..."
2,1502439,2015-08-04T12:00:00-05:00,Outros,Violação da Lei da Verdade em Empréstimos/Regu...
3,2834040,2018-03-05T12:00:00-05:00,Outros,A edição do JP Morgan Chase envolve dinheiro e...
4,2125959,2016-09-22T12:00:00-05:00,Serviços de conta bancária,Meu marido recebeu um visto de recompensa do A...


# Carregar a biblioteca spacy

In [5]:
nlp = spacy.load('pt_core_news_sm')


# Função de limpeza de texto

In [6]:
def limpar_textos(texto):
  texto = re.sub(r'[,.:;]', '', texto)
  texto = re.sub(r'\n', ' ', texto)
  texto = re.sub(r'[xxxx]', '', texto)
  texto = re.sub(r'[XXXX]', '', texto)
  texto = re.sub(r'\{.*?\}', '', texto)
  texto = re.sub(r'[xx/xx/xxxx]', '', texto)
  texto = re.sub('\\s+', ' ', texto)
  texto = re.sub(u'[^a-zA-ZéúíóáÉÚÍÓÁèùìòàÈÙÌÒÀõãñÕÃÑêûîôâÊÛÎÔÂëÿüïöäËYÜÏÖÄçÇ ]', '', texto)
  texto = texto.strip().lower()
  return texto

# Função de Lematização de texto

In [7]:
def lematiza_texto(texto: str):
  sent = []
  doc = nlp(texto)
  for word in doc:
    sent.append(word.lemma_)
  return " ".join(sent)

# Aplicando limpeza e lematização do texto

In [8]:
df["text_lemma"] = df.descricao_reclamacao.apply(limpar_textos).apply(lematiza_texto)

# Separação de dados entre treino e teste

In [9]:
df_treino, df_teste = train_test_split(df, 
                                       test_size=0.25, 
                                       random_state=42)

#Execução do modelo campeão
Utilizamos stopwords do NLTK, vetor combinando unigrama e bigrama, e modelo de Regressão Logística

In [10]:
nltk.download('stopwords')

stops_nltk = nltk.corpus.stopwords.words('portuguese')

vetor_tfid = TfidfVectorizer(ngram_range=(1,2), use_idf=True, stop_words=list(stops_nltk), norm='l2')
vetor_tfid.fit(df_treino.text_lemma)
text_vect_treino = vetor_tfid.transform(df_treino.text_lemma)

model = LogisticRegression(random_state=42, class_weight='balanced')
model.fit(text_vect_treino, df_treino["categoria"])

text_vect_test = vetor_tfid.transform(df_teste.text_lemma)
pred = model.predict(text_vect_test)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Medida de avaliação
Utilizamos o F1 Score como medida de avaliação do modelo

In [11]:
f1_score(df_teste['categoria'], pred, average='weighted')

0.8981898968915679